In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [4]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-6dq_6_jn
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-6dq_6_jn
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4287 sha256=1ac75ec62f8856b35ca50837d2ba2a90ef18d064f7f13f69feb74639505f4169
  Stored in directory: /tmp/pip-ephem-wheel-cache-h3ud0b6o/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin


In [5]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [25]:
%%cu
#include<stdio.h>
#include<cuda.h>
__global__ void vectorAddition(int *x,int *y, int *z)
{
    int id=blockIdx.x * blockDim.x + threadIdx.x;
 
    /* blockIdx.x gives the respective block id which starts from 0 */
    /* threadIdx.x gives the respective thread id which starts from 0 */
    /* blockDim.x gives the dimension of block i.e. number of threads in one block */
 
    z[id]=x[id]+y[id]; 
    printf("Thread %d and Block %d : %d + %d = %d\n", threadIdx.x, blockIdx.x, x[id], y[id], z[id] );
}
int main()
{
    int a[6] = {10, 20, 45, 32, 10, 21};
    int b[6] = {5, 6, 3, 51, 44, 10};
    int c[6];
    int *d,*e,*f;
    int i;
    /* printf("\n Enter six elements of first array\n");
     for(i=0;i<6;i++)
     {
         scanf("%d",&a[i]);
     }
     printf("\n Enter six elements of second array\n");
         for(i=0;i<6;i++)
         {
             scanf("%d",&b[i]);
         }
    */
 
  /* cudaMalloc() allocates memory from Global memory on GPU */
    cudaMalloc((void **)&d,6*sizeof(int));
    cudaMalloc((void **)&e,6*sizeof(int));
    cudaMalloc((void **)&f,6*sizeof(int));
 

 /* cudaMemcpy() copies the contents from destination to source. Here destination is GPU(d,e) and source is CPU(a,b) */
 cudaMemcpy(d,a,6*sizeof(int),cudaMemcpyHostToDevice);
 cudaMemcpy(e,b,6*sizeof(int),cudaMemcpyHostToDevice);

/* call to kernel. Here 2 is number of blocks, 3 is the number of threads per block and d,e,f are the arguments */ 
    vectorAddition<<<2,3>>>(d,e,f);
 
 cudaMemcpy(c,f,6*sizeof(int),cudaMemcpyDeviceToHost);
    printf("\nSum of two arrays:\n ");
    for(i=0;i<6;i++)
    {
        printf("%d\t",c[i]);
    }
    cudaFree(d);
    cudaFree(e);
    cudaFree(f);
    return 0;
}


Thread 0 and Block 1 : 32 + 51 = 83
Thread 1 and Block 1 : 10 + 44 = 54
Thread 2 and Block 1 : 21 + 10 = 31
Thread 0 and Block 0 : 10 + 5 = 15
Thread 1 and Block 0 : 20 + 6 = 26
Thread 2 and Block 0 : 45 + 3 = 48

Sum of two arrays:
 15	26	48	83	54	31	
